# 🔬 Inference Pipeline: Production-Ready Predictions

This notebook demonstrates the complete inference pipeline for making predictions with trained RL agents.

**Topics covered:**
- Single observation inference
- Batch inference
- Risk management
- Performance monitoring
- Production deployment patterns

**Time required:** ~20 minutes

## 1. Setup

In [13]:
# Add project root to Python path
import sys
from pathlib import Path

# Get the project root (parent of notebooks directory)
project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"✓ Project root: {project_root}")
print(f"✓ Python path configured")

✓ Project root: e:\Derivative_Hedging_RL
✓ Python path configured


In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import importlib

# Inference pipeline (reload to get latest changes)
from src.inference import pipeline as pipeline_module
importlib.reload(pipeline_module)
from src.inference.pipeline import InferencePipeline
from src.inference.data_loader import DataLoader
from src.inference.preprocessor import DataPreprocessor
from src.inference.postprocessor import PostProcessor

# For comparison
from src.agents.trainer import AgentTrainer
from src.agents.config import get_config, ENV_CONFIGS

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

print("✓ Imports successful!")
print("✓ Pipeline module reloaded with latest changes")

✓ Imports successful!
✓ Pipeline module reloaded with latest changes


## 2. Train a Model (or Load Existing)

For this demo, we'll train a quick model. In production, you'd load a pre-trained model.

In [15]:
# Check if we have a trained model
model_path = Path("models/notebook_inference/ppo_quick.zip")

if not model_path.exists():
    print("No existing model found. Training a new one...")
    print("This will take ~5 minutes")
    print("-" * 80)
    
    # Train a model
    trainer = AgentTrainer(
        agent_type="PPO",
        env_config=ENV_CONFIGS["medium"],
        output_dir="models/notebook_inference",
        seed=42
    )
    
    agent = trainer.quick_train(
        agent_config=get_config("PPO", "fast_learning"),
        total_timesteps=50000,
    )
    
    # Save it
    agent.save("models/notebook_inference/ppo_quick")
    print("\n✓ Model trained and saved!")
else:
    print(f"✓ Using existing model: {model_path}")

✓ Using existing model: models\notebook_inference\ppo_quick.zip


## 3. Initialize Inference Pipeline

The InferencePipeline class handles the entire workflow:
**Data → Load → Clean → Pre-Process → Inference → Post-Process**

**Important:** The pipeline needs to know the environment configuration that was used during training. The observation space must match between training and inference. 

- Models trained with `quick_train` on "medium" config: use `env_config=ENV_CONFIGS["medium"]`
- Models trained with `train_with_curriculum`: use `env_config=ENV_CONFIGS["hard"]` (final stage)

In [16]:
# Initialize pipeline
# Note: Model was trained with 'medium' config (n_steps=100)
# Must pass matching env_config for correct observation space
pipeline = InferencePipeline(
    model_path="models/notebook_inference/ppo_quick.zip",
    model_type="PPO",
    env_config=ENV_CONFIGS["medium"],  # Match training config
    apply_risk_limits=True,
    max_hedge_ratio=2.0,
    log_predictions=True,
)

# Get model info
model_info = pipeline.get_model_info()

print("Inference Pipeline Initialized")
print("=" * 80)
print(f"Model Type: {model_info['model_type']}")
print(f"Model Path: {model_info['model_path']}")
print(f"Environment Config: medium (n_steps=100)")
print(f"Risk Limits Enabled: {model_info['risk_limits_enabled']}")
print(f"Max Hedge Ratio: {model_info['max_hedge_ratio']}")
print("=" * 80)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Inference Pipeline Initialized
Model Type: PPO
Model Path: models\notebook_inference\ppo_quick.zip
Environment Config: medium (n_steps=100)
Risk Limits Enabled: True
Max Hedge Ratio: 2.0


e:\Derivative_Hedging_RL\.venv\Lib\site-packages\gymnasium\spaces\box.py:236: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
e:\Derivative_Hedging_RL\.venv\Lib\site-packages\gymnasium\spaces\box.py:306: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(
e:\Derivative_Hedging_RL\.venv\Lib\site-packages\stable_baselines3\common\policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


## 4. Single Observation Inference

Let's make a prediction for a single market condition.

In [17]:
# Market scenario: Slightly in-the-money call option
result = pipeline.predict_single(
    spot_price=105.0,        # Current stock price
    strike=100.0,            # Strike price
    time_to_maturity=0.25,   # 3 months to expiration
    risk_free_rate=0.05,     # 5% risk-free rate
    volatility=0.25,         # 25% implied volatility
    option_type="call",
    current_hedge=0.5,       # Currently 50% hedged
    deterministic=True,
)

print("\n" + "=" * 80)
print("SINGLE PREDICTION RESULT")
print("=" * 80)
print(f"\nMarket Conditions:")
print(f"  Spot Price: ${result['metadata']['features_shape']}")
print(f"  Moneyness: {105/100:.2f} (in-the-money)")
print(f"  Time to Expiration: 3 months")
print(f"  Volatility: 25%")

print(f"\nPrediction:")
print(f"  Status: {result['status']}")
print(f"  Target Hedge Ratio: {result['prediction']['target_hedge_ratio']:.4f}")
print(f"  Shares to Trade: {result['prediction']['shares_to_trade']:.4f}")
print(f"  Trade Value: ${result['prediction']['trade_value']:.2f}")
print(f"  Confidence: {result['confidence']:.2%}")
print(f"  Inference Time: {result['metadata']['inference_time_ms']:.2f} ms")

# Interpret
current_hedge = 0.5
target_hedge = result['prediction']['target_hedge_ratio']
action = "INCREASE" if target_hedge > current_hedge else "DECREASE"
print(f"\n📊 Action: {action} hedge position")

Failed to calculate Greeks: type object 'BlackScholesModel' has no attribute 'delta'



SINGLE PREDICTION RESULT

Market Conditions:
  Spot Price: $(11,)
  Moneyness: 1.05 (in-the-money)
  Time to Expiration: 3 months
  Volatility: 25%

Prediction:
  Status: success
  Target Hedge Ratio: -0.6364
  Shares to Trade: -1.1364
  Trade Value: $-119.32
  Confidence: 100.00%
  Inference Time: 2.44 ms

📊 Action: DECREASE hedge position


## 5. Batch Inference: Multiple Scenarios

Let's create a batch of different market scenarios and run inference on all of them.

In [20]:
# Create diverse test scenarios with different market conditions
np.random.seed(42)
n_samples = 200

# Generate diverse scenarios
scenarios = []

# 1. Standard scenarios (40%)
n_standard = int(n_samples * 0.4)
scenarios.append(pd.DataFrame({
    'spot_price': np.random.uniform(95, 105, n_standard),
    'strike': 100.0,
    'time_to_maturity': np.random.uniform(0.1, 0.5, n_standard),
    'risk_free_rate': 0.05,
    'volatility': np.random.uniform(0.18, 0.28, n_standard),
    'option_type': np.random.choice(['call', 'put'], n_standard),
    'current_hedge': np.random.uniform(0.3, 0.7, n_standard),
    'scenario_type': 'standard',
}))

# 2. Out-of-the-money scenarios (20%)
n_otm = int(n_samples * 0.2)
otm_spots = np.concatenate([
    np.random.uniform(85, 95, n_otm//2),  # OTM calls
    np.random.uniform(105, 115, n_otm//2),  # OTM puts
])
otm_types = ['call'] * (n_otm//2) + ['put'] * (n_otm//2)
scenarios.append(pd.DataFrame({
    'spot_price': otm_spots,
    'strike': 100.0,
    'time_to_maturity': np.random.uniform(0.1, 0.5, n_otm),
    'risk_free_rate': 0.05,
    'volatility': np.random.uniform(0.20, 0.30, n_otm),
    'option_type': otm_types,
    'current_hedge': np.random.uniform(0.1, 0.4, n_otm),
    'scenario_type': 'out_of_money',
}))

# 3. In-the-money scenarios (20%)
n_itm = int(n_samples * 0.2)
itm_spots = np.concatenate([
    np.random.uniform(105, 115, n_itm//2),  # ITM calls
    np.random.uniform(85, 95, n_itm//2),  # ITM puts
])
itm_types = ['call'] * (n_itm//2) + ['put'] * (n_itm//2)
scenarios.append(pd.DataFrame({
    'spot_price': itm_spots,
    'strike': 100.0,
    'time_to_maturity': np.random.uniform(0.1, 0.5, n_itm),
    'risk_free_rate': 0.05,
    'volatility': np.random.uniform(0.20, 0.30, n_itm),
    'option_type': itm_types,
    'current_hedge': np.random.uniform(0.5, 0.9, n_itm),
    'scenario_type': 'in_the_money',
}))

# 4. High volatility regime (10%)
n_high_vol = int(n_samples * 0.1)
scenarios.append(pd.DataFrame({
    'spot_price': np.random.uniform(90, 110, n_high_vol),
    'strike': 100.0,
    'time_to_maturity': np.random.uniform(0.1, 0.5, n_high_vol),
    'risk_free_rate': 0.05,
    'volatility': np.random.uniform(0.35, 0.50, n_high_vol),
    'option_type': np.random.choice(['call', 'put'], n_high_vol),
    'current_hedge': np.random.uniform(0.3, 0.7, n_high_vol),
    'scenario_type': 'high_volatility',
}))

# 5. Near expiration (10%)
n_near_exp = n_samples - n_standard - n_otm - n_itm - n_high_vol
scenarios.append(pd.DataFrame({
    'spot_price': np.random.uniform(95, 105, n_near_exp),
    'strike': 100.0,
    'time_to_maturity': np.random.uniform(0.01, 0.08, n_near_exp),  # 3-20 days
    'risk_free_rate': 0.05,
    'volatility': np.random.uniform(0.20, 0.35, n_near_exp),
    'option_type': np.random.choice(['call', 'put'], n_near_exp),
    'current_hedge': np.random.uniform(0.3, 0.7, n_near_exp),
    'scenario_type': 'near_expiration',
}))

# Combine all scenarios
test_data = pd.concat(scenarios, ignore_index=True)

# Add moneyness for analysis
test_data['moneyness'] = test_data['spot_price'] / test_data['strike']

# Save to CSV
test_data.to_csv("data/test_scenarios.csv", index=False)

print(f"✓ Created {len(test_data)} diverse test scenarios")
print("\nScenario Distribution:")
print(test_data['scenario_type'].value_counts())
print("\nOption Type Distribution:")
print(test_data['option_type'].value_counts())
print("\nFirst 5 scenarios:")
print(test_data[['spot_price', 'strike', 'time_to_maturity', 'volatility', 
                  'option_type', 'scenario_type', 'moneyness']].head())

✓ Created 200 diverse test scenarios

Scenario Distribution:
scenario_type
standard           80
out_of_money       40
in_the_money       40
high_volatility    20
near_expiration    20
Name: count, dtype: int64

Option Type Distribution:
option_type
call    105
put      95
Name: count, dtype: int64

First 5 scenarios:
   spot_price  strike  time_to_maturity  volatility option_type scenario_type  \
0   98.745401   100.0          0.445241    0.216778         put      standard   
1  104.507143   100.0          0.349319    0.243231         put      standard   
2  102.319939   100.0          0.232359    0.243353        call      standard   
3  100.986585   100.0          0.125423    0.233577         put      standard   
4   96.560186   100.0          0.224393    0.189029        call      standard   

   moneyness  
0   0.987454  
1   1.045071  
2   1.023199  
3   1.009866  
4   0.965602  


In [21]:
# Reload modules to get the latest fixes
import importlib
from src.inference import pipeline as pipeline_module
from src.inference import preprocessor as preprocessor_module

importlib.reload(preprocessor_module)
importlib.reload(pipeline_module)

from src.inference.pipeline import InferencePipeline

# Recreate pipeline with reloaded modules
pipeline = InferencePipeline(
    model_path="models/notebook_inference/ppo_quick.zip",
    model_type="PPO",
    env_config=ENV_CONFIGS["medium"],
    apply_risk_limits=True,
    max_hedge_ratio=2.0,
    log_predictions=True,
)

print("✓ Modules reloaded with latest fixes")
print("✓ Pipeline recreated")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
✓ Modules reloaded with latest fixes
✓ Pipeline recreated


e:\Derivative_Hedging_RL\.venv\Lib\site-packages\gymnasium\spaces\box.py:236: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
e:\Derivative_Hedging_RL\.venv\Lib\site-packages\gymnasium\spaces\box.py:306: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(
e:\Derivative_Hedging_RL\.venv\Lib\site-packages\stable_baselines3\common\policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


In [ ]:
# Run batch inference with performance tracking
print("\nRunning batch inference...")
print("=" * 80)

start_time = time.time()

results_df = pipeline.predict_batch(
    data="data/test_scenarios.csv",
    deterministic=True,
    save_results="results/batch_predictions.csv",
)

inference_time = time.time() - start_time

# Calculate Black-Scholes Delta for comparison
from scipy.stats import norm

def black_scholes_delta(S, K, T, r, sigma, option_type='call'):
    """Calculate Black-Scholes delta"""
    if T <= 0:
        # At expiration
        if option_type == 'call':
            return 1.0 if S > K else 0.0
        else:
            return -1.0 if S < K else 0.0
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    if option_type == 'call':
        return norm.cdf(d1)
    else:
        return norm.cdf(d1) - 1.0

# Add Black-Scholes delta as benchmark
results_df['bs_delta'] = results_df.apply(
    lambda row: black_scholes_delta(
        row['spot_price'], row['strike'], row['time_to_maturity'],
        row['risk_free_rate'], row['volatility'], row['option_type']
    ), axis=1
)

# Calculate deviation from Black-Scholes
results_df['delta_deviation'] = results_df['target_hedge_ratio'] - results_df['bs_delta']
results_df['abs_delta_deviation'] = np.abs(results_df['delta_deviation'])

print(f"\n✓ Batch inference complete!")
print(f"  Total time: {inference_time:.2f} seconds")
print(f"  Throughput: {len(results_df)/inference_time:.1f} predictions/second")
print(f"  Average time per prediction: {inference_time/len(results_df)*1000:.2f} ms")
print(f"  Results saved to: results/batch_predictions.csv")

# Performance breakdown
print(f"\nPerformance Summary:")
print(f"  Total Scenarios: {len(results_df)}")
print(f"  Average Confidence: {results_df['confidence'].mean():.2%}")
print(f"  Min Confidence: {results_df['confidence'].min():.2%}")
print(f"  Max Confidence: {results_df['confidence'].max():.2%}")

# Benchmark comparison
print(f"\nBenchmark Comparison (vs Black-Scholes Delta):")
print(f"  Mean Absolute Deviation: {results_df['abs_delta_deviation'].mean():.4f}")
print(f"  Median Absolute Deviation: {results_df['abs_delta_deviation'].median():.4f}")
print(f"  Max Absolute Deviation: {results_df['abs_delta_deviation'].max():.4f}")
print(f"  Correlation with BS Delta: {results_df['target_hedge_ratio'].corr(results_df['bs_delta']):.4f}")

# Display sample results
print("\n" + "=" * 80)
print("SAMPLE RESULTS (First 10)")
print("=" * 80)
display_cols = ['spot_price', 'option_type', 'scenario_type', 'time_to_maturity', 
                'volatility', 'target_hedge_ratio', 'bs_delta', 'delta_deviation', 'confidence']
print(results_df[display_cols].head(10).to_string(index=False))


Running batch inference...

✓ Batch inference complete!
  Total time: 0.11 seconds
  Throughput: 1878.0 predictions/second
  Average time per prediction: 0.53 ms
  Results saved to: results/batch_predictions.csv

Performance Summary:
  Total Scenarios: 200


KeyError: 'status'

### 5.1 Scenario-Specific Analysis

Let's analyze how the model performs across different scenario types.

In [ ]:
# Group by scenario type and calculate metrics
scenario_analysis = results_df.groupby('scenario_type').agg({
    'target_hedge_ratio': ['mean', 'std', 'min', 'max'],
    'bs_delta': ['mean'],
    'abs_delta_deviation': ['mean', 'max'],
    'confidence': ['mean', 'min'],
    'trade_value': lambda x: np.abs(x).mean(),
}).round(4)

scenario_analysis.columns = ['_'.join(col).strip() for col in scenario_analysis.columns.values]
scenario_analysis = scenario_analysis.rename(columns={
    'target_hedge_ratio_mean': 'Avg_Hedge_Ratio',
    'target_hedge_ratio_std': 'Std_Hedge_Ratio',
    'target_hedge_ratio_min': 'Min_Hedge_Ratio',
    'target_hedge_ratio_max': 'Max_Hedge_Ratio',
    'bs_delta_mean': 'Avg_BS_Delta',
    'abs_delta_deviation_mean': 'Avg_Deviation',
    'abs_delta_deviation_max': 'Max_Deviation',
    'confidence_mean': 'Avg_Confidence',
    'confidence_min': 'Min_Confidence',
    'trade_value_<lambda>': 'Avg_Trade_Size',
})

print("Performance by Scenario Type")
print("=" * 100)
print(scenario_analysis.to_string())

# Visualize scenario comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Average hedge ratio by scenario
scenario_summary = results_df.groupby('scenario_type')['target_hedge_ratio'].mean().sort_values()
axes[0, 0].barh(scenario_summary.index, scenario_summary.values, color='steelblue')
axes[0, 0].set_xlabel('Average Hedge Ratio')
axes[0, 0].set_title('Average Hedge Ratio by Scenario Type')
axes[0, 0].grid(True, alpha=0.3, axis='x')

# 2. Deviation from Black-Scholes by scenario
scenario_dev = results_df.groupby('scenario_type')['abs_delta_deviation'].mean().sort_values()
axes[0, 1].barh(scenario_dev.index, scenario_dev.values, color='coral')
axes[0, 1].set_xlabel('Mean Absolute Deviation from BS Delta')
axes[0, 1].set_title('Deviation from Black-Scholes by Scenario')
axes[0, 1].grid(True, alpha=0.3, axis='x')

# 3. Confidence by scenario type
scenario_conf = results_df.groupby('scenario_type')['confidence'].mean().sort_values()
axes[1, 0].barh(scenario_conf.index, scenario_conf.values, color='mediumseagreen')
axes[1, 0].set_xlabel('Average Confidence')
axes[1, 0].set_title('Model Confidence by Scenario Type')
axes[1, 0].grid(True, alpha=0.3, axis='x')

# 4. Distribution of hedge ratios by scenario
scenario_types = results_df['scenario_type'].unique()
positions = np.arange(len(scenario_types))
box_data = [results_df[results_df['scenario_type'] == st]['target_hedge_ratio'].values 
            for st in scenario_types]
bp = axes[1, 1].boxplot(box_data, positions=positions, patch_artist=True, labels=scenario_types)
for patch in bp['boxes']:
    patch.set_facecolor('lightblue')
axes[1, 1].set_ylabel('Hedge Ratio')
axes[1, 1].set_title('Hedge Ratio Distribution by Scenario')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('results/scenario_analysis.png', dpi=150)
plt.show()

print("\n✓ Scenario analysis complete")

### 5.2 Pre-Defined Market Scenarios

Test the model on specific real-world scenarios like market crashes, low volatility periods, etc.

In [ ]:
# Define real-world scenario templates
scenario_templates = {
    'market_crash': {
        'name': 'Market Crash (-20%)',
        'spot_price': 80.0,
        'strike': 100.0,
        'time_to_maturity': 0.25,
        'risk_free_rate': 0.05,
        'volatility': 0.60,  # Very high volatility
        'option_type': 'call',
        'current_hedge': 0.5,
        'description': 'Sudden market drop with volatility spike'
    },
    'bull_run': {
        'name': 'Strong Bull Market (+15%)',
        'spot_price': 115.0,
        'strike': 100.0,
        'time_to_maturity': 0.25,
        'risk_free_rate': 0.05,
        'volatility': 0.18,  # Low volatility
        'option_type': 'call',
        'current_hedge': 0.7,
        'description': 'Strong upward trend with low volatility'
    },
    'low_vol_regime': {
        'name': 'Low Volatility Period',
        'spot_price': 100.0,
        'strike': 100.0,
        'time_to_maturity': 0.50,
        'risk_free_rate': 0.05,
        'volatility': 0.10,  # Extremely low volatility
        'option_type': 'call',
        'current_hedge': 0.5,
        'description': 'Calm markets with minimal movement'
    },
    'pre_earnings': {
        'name': 'Pre-Earnings Announcement',
        'spot_price': 102.0,
        'strike': 100.0,
        'time_to_maturity': 0.05,  # 2 weeks
        'risk_free_rate': 0.05,
        'volatility': 0.45,  # High implied vol
        'option_type': 'call',
        'current_hedge': 0.6,
        'description': 'High uncertainty before major announcement'
    },
    'expiration_week': {
        'name': 'Expiration Week (ATM)',
        'spot_price': 100.0,
        'strike': 100.0,
        'time_to_maturity': 0.0192,  # 5 days
        'risk_free_rate': 0.05,
        'volatility': 0.30,
        'option_type': 'call',
        'current_hedge': 0.5,
        'description': 'Close to expiration, at-the-money'
    },
    'deep_otm_put': {
        'name': 'Deep OTM Protective Put',
        'spot_price': 110.0,
        'strike': 100.0,
        'time_to_maturity': 0.25,
        'risk_free_rate': 0.05,
        'volatility': 0.25,
        'option_type': 'put',
        'current_hedge': 0.1,
        'description': 'Far out-of-the-money put protection'
    },
}

# Run predictions on all scenario templates
print("Testing Pre-Defined Market Scenarios")
print("=" * 100)

template_results = []

for key, scenario in scenario_templates.items():
    result = pipeline.predict_single(
        spot_price=scenario['spot_price'],
        strike=scenario['strike'],
        time_to_maturity=scenario['time_to_maturity'],
        risk_free_rate=scenario['risk_free_rate'],
        volatility=scenario['volatility'],
        option_type=scenario['option_type'],
        current_hedge=scenario['current_hedge'],
        deterministic=True,
    )
    
    # Calculate BS Delta for comparison
    bs_delta = black_scholes_delta(
        scenario['spot_price'], scenario['strike'], scenario['time_to_maturity'],
        scenario['risk_free_rate'], scenario['volatility'], scenario['option_type']
    )
    
    template_results.append({
        'Scenario': scenario['name'],
        'Description': scenario['description'],
        'Spot': scenario['spot_price'],
        'Strike': scenario['strike'],
        'TTM': scenario['time_to_maturity'],
        'Vol': scenario['volatility'],
        'Type': scenario['option_type'],
        'RL_Hedge': result['prediction']['target_hedge_ratio'],
        'BS_Delta': bs_delta,
        'Deviation': result['prediction']['target_hedge_ratio'] - bs_delta,
        'Confidence': result['confidence'],
        'Action': result['prediction']['shares_to_trade'],
    })

# Display results
template_df = pd.DataFrame(template_results)

print("\n" + "=" * 100)
print("SCENARIO TEMPLATE PREDICTIONS")
print("=" * 100)
for idx, row in template_df.iterrows():
    print(f"\n{idx+1}. {row['Scenario']}")
    print(f"   {row['Description']}")
    print(f"   Market: S=${row['Spot']:.0f}, K=${row['Strike']:.0f}, "
          f"T={row['TTM']*252:.0f}d, σ={row['Vol']:.1%}, Type={row['Type']}")
    print(f"   RL Hedge: {row['RL_Hedge']:.4f} | BS Delta: {row['BS_Delta']:.4f} | "
          f"Deviation: {row['Deviation']:+.4f}")
    print(f"   Confidence: {row['Confidence']:.1%} | Action: {row['Action']:+.4f} shares")

# Visualize comparison
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(template_df))
width = 0.35

bars1 = ax.bar(x - width/2, template_df['RL_Hedge'], width, label='RL Agent', color='steelblue')
bars2 = ax.bar(x + width/2, template_df['BS_Delta'], width, label='Black-Scholes', color='coral')

ax.set_xlabel('Scenario')
ax.set_ylabel('Hedge Ratio')
ax.set_title('RL Agent vs Black-Scholes Delta: Pre-Defined Scenarios')
ax.set_xticks(x)
ax.set_xticklabels(template_df['Scenario'], rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('results/scenario_templates_comparison.png', dpi=150)
plt.show()

print("\n✓ Scenario template analysis complete")

### 5.3 Export Results in Multiple Formats

Export predictions for downstream systems and reporting.

In [ ]:
# Export batch results in multiple formats
import json
from datetime import datetime

export_dir = Path("results/exports")
export_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# 1. CSV (already saved, but let's create a summary version)
summary_cols = ['spot_price', 'strike', 'option_type', 'time_to_maturity', 'volatility',
                'target_hedge_ratio', 'bs_delta', 'delta_deviation', 'confidence', 
                'shares_to_trade', 'trade_value', 'scenario_type']
results_df[summary_cols].to_csv(f"{export_dir}/predictions_summary_{timestamp}.csv", index=False)
print(f"✓ CSV exported: {export_dir}/predictions_summary_{timestamp}.csv")

# 2. Excel with multiple sheets
try:
    with pd.ExcelWriter(f"{export_dir}/predictions_{timestamp}.xlsx", engine='openpyxl') as writer:
        # Full results
        results_df.to_excel(writer, sheet_name='Full_Results', index=False)
        
        # Summary statistics by scenario
        scenario_stats = results_df.groupby('scenario_type').agg({
            'target_hedge_ratio': ['mean', 'std', 'min', 'max'],
            'confidence': ['mean', 'min'],
            'abs_delta_deviation': 'mean',
        }).round(4)
        scenario_stats.to_excel(writer, sheet_name='Scenario_Stats')
        
        # High confidence trades only
        high_conf = results_df[results_df['confidence'] >= 0.8]
        high_conf[summary_cols].to_excel(writer, sheet_name='High_Confidence', index=False)
        
    print(f"✓ Excel exported: {export_dir}/predictions_{timestamp}.xlsx")
except ImportError:
    print("⚠ Excel export requires openpyxl: pip install openpyxl")

# 3. JSON for API integration
json_results = []
for idx, row in results_df.iterrows():
    json_results.append({
        'prediction_id': idx,
        'timestamp': datetime.now().isoformat(),
        'input': {
            'spot_price': float(row['spot_price']),
            'strike': float(row['strike']),
            'time_to_maturity': float(row['time_to_maturity']),
            'volatility': float(row['volatility']),
            'option_type': row['option_type'],
            'current_hedge': float(row['current_hedge']),
        },
        'prediction': {
            'target_hedge_ratio': float(row['target_hedge_ratio']),
            'shares_to_trade': float(row['shares_to_trade']),
            'trade_value': float(row['trade_value']),
            'confidence': float(row['confidence']),
        },
        'benchmark': {
            'black_scholes_delta': float(row['bs_delta']),
            'deviation': float(row['delta_deviation']),
        },
        'metadata': {
            'scenario_type': row['scenario_type'],
            'moneyness': float(row['moneyness']),
        }
    })

with open(f"{export_dir}/predictions_{timestamp}.json", 'w') as f:
    json.dump({
        'export_time': datetime.now().isoformat(),
        'model_type': 'PPO',
        'total_predictions': len(json_results),
        'predictions': json_results[:10]  # Save first 10 for demo (full export would be large)
    }, f, indent=2)
print(f"✓ JSON exported: {export_dir}/predictions_{timestamp}.json (sample of 10)")

# 4. Trading instruction format (for execution systems)
trading_instructions = []
for idx, row in results_df.iterrows():
    if row['confidence'] >= 0.75:  # Only high confidence trades
        instruction = {
            'order_id': f"RL_{timestamp}_{idx:04d}",
            'security': f"{row['option_type'].upper()}_{row['strike']:.0f}_{row['time_to_maturity']*252:.0f}D",
            'action': 'BUY' if row['shares_to_trade'] > 0 else 'SELL',
            'quantity': abs(float(row['shares_to_trade'])),
            'reason': f"RL_hedge_adjustment",
            'confidence': float(row['confidence']),
            'current_delta': float(row['current_hedge']),
            'target_delta': float(row['target_hedge_ratio']),
        }
        trading_instructions.append(instruction)

with open(f"{export_dir}/trading_instructions_{timestamp}.json", 'w') as f:
    json.dump({
        'generated_at': datetime.now().isoformat(),
        'total_instructions': len(trading_instructions),
        'instructions': trading_instructions
    }, f, indent=2)

print(f"✓ Trading instructions exported: {export_dir}/trading_instructions_{timestamp}.json")
print(f"\nExport Summary:")
print(f"  Total predictions: {len(results_df)}")
print(f"  High confidence (≥75%): {len(trading_instructions)}")
print(f"  Export directory: {export_dir}")

### 5.4 Batch Inference Summary

**What we accomplished in this section:**

✓ **Diverse Scenarios**: Generated 200 test cases across 5 market regimes
- Standard conditions (40%)
- Out-of-the-money (20%)
- In-the-money (20%)
- High volatility (10%)
- Near expiration (10%)

✓ **Benchmark Comparisons**: Calculated Black-Scholes delta for all predictions
- Mean absolute deviation from BS
- Correlation analysis
- Identification of significant deviations

✓ **Performance Profiling**: Measured inference speed and throughput
- Predictions per second
- Average latency per prediction
- Confidence distribution

✓ **Scenario Analysis**: Analyzed performance by market regime
- Aggregated statistics by scenario type
- Visualization of differences
- Risk-adjusted metrics

✓ **Real-World Templates**: Pre-defined scenarios for common situations
- Market crash
- Bull market
- Low volatility
- Pre-earnings
- Expiration week
- Deep OTM protection

✓ **Multi-Format Export**: Ready for production integration
- CSV for spreadsheets
- Excel with multiple sheets
- JSON for APIs
- Trading instructions format

**Next**: Advanced visualization and risk management analysis

## 6. Advanced Visualization & Analysis

Comprehensive visualizations showing RL predictions vs Black-Scholes benchmarks across market conditions.

In [ ]:
# Comprehensive visualization of predictions
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# 1. Hedge ratio vs Moneyness (RL and BS)
moneyness = results_df['moneyness']
axes[0, 0].scatter(moneyness, results_df['target_hedge_ratio'], 
                   c=results_df['confidence'], cmap='viridis', alpha=0.6, 
                   label='RL Agent', s=50)
axes[0, 0].scatter(moneyness, results_df['bs_delta'], 
                   alpha=0.3, color='red', marker='x', s=30, label='BS Delta')
axes[0, 0].set_xlabel('Moneyness (S/K)')
axes[0, 0].set_ylabel('Hedge Ratio')
axes[0, 0].set_title('Hedge Ratio vs Moneyness')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)
cbar1 = plt.colorbar(axes[0, 0].collections[0], ax=axes[0, 0])
cbar1.set_label('Confidence')

# 2. Hedge ratio vs Time to Maturity
axes[0, 1].scatter(results_df['time_to_maturity'], results_df['target_hedge_ratio'],
                   c=results_df['confidence'], cmap='viridis', alpha=0.6, s=50,
                   label='RL Agent')
axes[0, 1].scatter(results_df['time_to_maturity'], results_df['bs_delta'],
                   alpha=0.3, color='red', marker='x', s=30, label='BS Delta')
axes[0, 1].set_xlabel('Time to Maturity (years)')
axes[0, 1].set_ylabel('Hedge Ratio')
axes[0, 1].set_title('Hedge Ratio vs Time to Maturity')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Hedge ratio vs Volatility
axes[0, 2].scatter(results_df['volatility'], results_df['target_hedge_ratio'],
                   c=results_df['confidence'], cmap='viridis', alpha=0.6, s=50,
                   label='RL Agent')
axes[0, 2].scatter(results_df['volatility'], results_df['bs_delta'],
                   alpha=0.3, color='red', marker='x', s=30, label='BS Delta')
axes[0, 2].set_xlabel('Volatility')
axes[0, 2].set_ylabel('Hedge Ratio')
axes[0, 2].set_title('Hedge Ratio vs Volatility')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

# 4. RL vs BS Delta (scatter)
axes[1, 0].scatter(results_df['bs_delta'], results_df['target_hedge_ratio'],
                   c=results_df['confidence'], cmap='plasma', alpha=0.6, s=50)
# Add diagonal line (perfect match)
min_val = min(results_df['bs_delta'].min(), results_df['target_hedge_ratio'].min())
max_val = max(results_df['bs_delta'].max(), results_df['target_hedge_ratio'].max())
axes[1, 0].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Match')
axes[1, 0].set_xlabel('Black-Scholes Delta')
axes[1, 0].set_ylabel('RL Agent Hedge Ratio')
axes[1, 0].set_title('RL Agent vs Black-Scholes')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)
cbar2 = plt.colorbar(axes[1, 0].collections[0], ax=axes[1, 0])
cbar2.set_label('Confidence')

# 5. Trade value distribution by option type
call_trades = results_df[results_df['option_type'] == 'call']['trade_value']
put_trades = results_df[results_df['option_type'] == 'put']['trade_value']
axes[1, 1].hist([call_trades, put_trades], bins=30, alpha=0.7, 
                label=['Calls', 'Puts'], color=['steelblue', 'coral'])
axes[1, 1].axvline(0, color='black', linestyle='--', linewidth=1)
axes[1, 1].set_xlabel('Trade Value ($)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Distribution of Trade Values by Option Type')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# 6. Deviation distribution
axes[1, 2].hist(results_df['delta_deviation'], bins=40, alpha=0.7, 
                edgecolor='black', color='mediumseagreen')
axes[1, 2].axvline(0, color='red', linestyle='--', linewidth=2, label='No Deviation')
axes[1, 2].axvline(results_df['delta_deviation'].mean(), color='blue', 
                   linestyle='--', linewidth=2, label=f'Mean: {results_df["delta_deviation"].mean():.4f}')
axes[1, 2].set_xlabel('Deviation from BS Delta')
axes[1, 2].set_ylabel('Frequency')
axes[1, 2].set_title('Distribution of Deviations from Black-Scholes')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/batch_analysis_enhanced.png', dpi=150)
plt.show()

print("✓ Enhanced batch analysis plots saved")
print(f"\n📊 Key Insights:")
print(f"  Correlation with BS: {results_df['target_hedge_ratio'].corr(results_df['bs_delta']):.3f}")
print(f"  Mean Deviation: {results_df['delta_deviation'].mean():+.4f}")
print(f"  Cases where |deviation| > 0.1: {(results_df['abs_delta_deviation'] > 0.1).sum()} ({(results_df['abs_delta_deviation'] > 0.1).mean():.1%})")

## 7. Generate Batch Report

The pipeline can generate comprehensive reports.

In [ ]:
# Generate report
report = pipeline.generate_batch_report(
    results_df=results_df,
    output_path="results/batch_inference_report.txt",
)

print(report)

## 8. Risk Management: Confidence-Based Filtering

In production, you might want to only execute high-confidence predictions.

In [ ]:
# Filter by confidence
confidence_threshold = 0.8
high_confidence = results_df[results_df['confidence'] >= confidence_threshold]
low_confidence = results_df[results_df['confidence'] < confidence_threshold]

print(f"Risk Management Analysis")
print("=" * 80)
print(f"Total Predictions: {len(results_df)}")
print(f"High Confidence (≥{confidence_threshold:.0%}): {len(high_confidence)} ({len(high_confidence)/len(results_df):.1%})")
print(f"Low Confidence (<{confidence_threshold:.0%}): {len(low_confidence)} ({len(low_confidence)/len(results_df):.1%})")

print(f"\nHigh Confidence Trades:")
print(f"  Total Trade Value: ${high_confidence['trade_value'].abs().sum():,.2f}")
print(f"  Average Hedge Ratio: {high_confidence['target_hedge_ratio'].mean():.4f}")

print(f"\nLow Confidence Trades (may want to skip):")
print(f"  Total Trade Value: ${low_confidence['trade_value'].abs().sum():,.2f}")
print(f"  Average Hedge Ratio: {low_confidence['target_hedge_ratio'].mean():.4f}")

# Visualize confidence distribution
plt.figure(figsize=(10, 6))
plt.hist(results_df['confidence'], bins=30, alpha=0.7, edgecolor='black')
plt.axvline(confidence_threshold, color='red', linestyle='--', linewidth=2, 
            label=f'Threshold ({confidence_threshold:.0%})')
plt.xlabel('Confidence Score')
plt.ylabel('Frequency')
plt.title('Confidence Score Distribution')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('results/confidence_distribution.png', dpi=150)
plt.show()

## 9. Inference Speed Benchmark

Let's test the inference throughput for production deployment planning.

In [ ]:
# Benchmark inference speed
print("Running inference speed benchmark...")
print("Testing with 1,000 predictions")
print("-" * 80)

benchmark_results = pipeline.benchmark_inference_speed(n_samples=1000)

print("\n" + "=" * 80)
print("BENCHMARK RESULTS")
print("=" * 80)
print(f"Number of Samples: {benchmark_results['n_samples']:,}")
print(f"Total Time: {benchmark_results['total_time_seconds']:.2f} seconds")
print(f"Throughput: {benchmark_results['samples_per_second']:.1f} predictions/second")
print(f"Latency: {benchmark_results['ms_per_sample']:.2f} ms/prediction")

print("\n📊 Production Capacity Estimate:")
print(f"  Per Minute: ~{benchmark_results['samples_per_second']*60:,.0f} predictions")
print(f"  Per Hour: ~{benchmark_results['samples_per_second']*3600:,.0f} predictions")
print(f"  Per Day: ~{benchmark_results['samples_per_second']*86400:,.0f} predictions")

## 10. Component-Level Testing

Let's test each pipeline component independently.

In [ ]:
# Test DataLoader
print("Testing DataLoader...")
loader = DataLoader()

loaded_data = loader.load_from_csv("data/test_scenarios.csv")
print(f"✓ Loaded {len(loaded_data)} rows")

realtime_obs = loader.load_realtime_observation(
    spot_price=105, strike=100, time_to_maturity=0.25,
    risk_free_rate=0.05, volatility=0.25
)
print(f"✓ Created real-time observation")

# Test Preprocessor
print("\nTesting Preprocessor...")
preprocessor = DataPreprocessor()

features = preprocessor.engineer_features(
    spot_price=105, strike=100, time_to_maturity=0.25,
    risk_free_rate=0.05, volatility=0.25
)
print(f"✓ Engineered features: shape={features.shape}")
print(f"  Features: {features}")

# Test PostProcessor
print("\nTesting PostProcessor...")
postprocessor = PostProcessor()

action = np.array([0.75])
processed = postprocessor.process_action(
    action=action,
    current_hedge=0.5,
    spot_price=105.0,
)
print(f"✓ Processed action:")
print(f"  Target hedge: {processed['target_hedge_ratio']:.4f}")
print(f"  Adjustment: {processed['hedge_adjustment']:.4f}")
print(f"  Shares to trade: {processed['shares_to_trade']:.4f}")

print("\n✓ All components working correctly!")

## 11. Production Deployment Patterns

Here are common patterns for deploying the inference pipeline in production.

In [ ]:
print("Production Deployment Patterns")
print("=" * 80)

patterns = """
1. BATCH PROCESSING (Recommended for start)
   - Run inference on all positions daily/hourly
   - Low latency requirements
   - Easy to implement and monitor
   
   Example:
   ```python
   # Scheduled job (e.g., cron)
   pipeline = InferencePipeline("models/production.zip")
   results = pipeline.predict_batch("data/current_positions.csv")
   results.to_csv("data/hedge_recommendations.csv")
   ```

2. REST API (For integration with trading systems)
   - Real-time inference on demand
   - Higher latency requirements
   - More complex infrastructure
   
   Example:
   ```python
   from fastapi import FastAPI
   app = FastAPI()
   pipeline = InferencePipeline("models/production.zip")
   
   @app.post("/predict")
   def predict(market_data: MarketData):
       result = pipeline.predict_single(**market_data.dict())
       return result
   ```

3. STREAMING (For high-frequency trading)
   - Continuous inference on market data stream
   - Very low latency requirements
   - Most complex infrastructure
   
   Example:
   ```python
   import kafka
   consumer = kafka.Consumer('market-data')
   producer = kafka.Producer('hedge-signals')
   pipeline = InferencePipeline("models/production.zip")
   
   for message in consumer:
       result = pipeline.predict_single(**message.value)
       producer.send(result)
   ```

4. MODEL SERVING (Using dedicated serving infrastructure)
   - TensorFlow Serving, TorchServe, or Seldon
   - Auto-scaling, monitoring, versioning built-in
   - Production-grade reliability
"""

print(patterns)

## 12. Summary and Best Practices

**Key Takeaways:**

1. **Inference Pipeline Benefits:**
   - Consistent preprocessing
   - Built-in risk management
   - Easy monitoring and logging
   - Production-ready

2. **Performance:**
   - ~100-1000 predictions/second on CPU
   - ~1000-10000 predictions/second on GPU
   - Latency: 1-10ms per prediction

3. **Risk Management:**
   - Use confidence scores to filter predictions
   - Apply position limits (max_hedge_ratio)
   - Monitor prediction quality

4. **Deployment:**
   - Start with batch processing
   - Move to API for real-time needs
   - Use model serving for scale

5. **Monitoring:**
   - Track inference speed
   - Monitor prediction distribution
   - Alert on anomalies

**Production Checklist:**
- [ ] Train and validate model thoroughly
- [ ] Test on out-of-sample data
- [ ] Set appropriate risk limits
- [ ] Implement monitoring
- [ ] Set up alerting
- [ ] Plan for model updates
- [ ] Document API/interface
- [ ] Load testing

**Next Steps:**
- Test with real market data
- Implement your deployment pattern
- Set up monitoring dashboard
- Create operational runbook